In [1]:
import os 
import matplotlib.pyplot as plt
from scipy import spatial
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

In [2]:
data_folder = 'data'
esg_filename = 'esg.csv'
esg_new_filename = 'esg_new.csv'
glove_file = 'glove-vectors/glove.6B.200d.txt'
glove_path = os.path.abspath(os.path.join(glove_file))
esg_path =  os.path.abspath(os.path.join(data_folder, esg_filename))
esg_new_path = os.path.abspath(os.path.join(data_folder, esg_new_filename))

In [3]:
def find_similar_words(emmbedes):

    nearest = sorted(emmbed_dict.keys(), key=lambda word: spatial.distance.euclidean(emmbed_dict[word], emmbedes))

    return nearest
    
emmbed_dict = {}
with open(glove_path) as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], 'float32')
        emmbed_dict[word] = vector

In [4]:
esg_seed = pd.read_csv(esg_path)
esg_seed

,E,S,G
0,biodiversity,social,governance
1,carbon,diversity,management
2,environment,equality,structure
3,emission,inclusion,compensation
4,pollution,fair,executive
5,sustainable,fairness,employee
6,waste,inequality,relations
7,Co2,equity,equitable
8,deforestation,welfare,pay
9,greenhouse,rights,NaN


In [42]:
esg_new = []#pd.DataFrame(columns=['E' ,'S' ,'G'])


for col in esg_seed.columns:

    word_list = []
    words = esg_seed[col].dropna()
    
    for word in words:
        try:
            word_list.append(find_similar_words(emmbed_dict[word.strip().lower()])[0:10])

        except AttributeError as e:
            print(e)

    # add back seed words
    word_list = np.concatenate((np.asarray(word_list).reshape(-1), np.asarray(esg_seed[col])))
    #esg_new[col] = np.unique(np.asarray(word_list).reshape(1,-1))
    word_list = word_list[np.where(word_list.astype(str) != 'nan')]
    esg_new.append(np.unique(word_list))

In [43]:
e = pd.DataFrame(data=esg_new[0])
s = pd.DataFrame(data=esg_new[1])
g = pd.DataFrame(data=esg_new[2])
esg_new = pd.concat([e,s,g], ignore_index=True, axis=1)
esg_new.columns = ['E', 'S', 'G']
esg_new

,E,S,G
0,Co2,accountability,accountability
1,achieve,activist,addition
2,achieving,advocates,afford
3,aim,animal,agreeable
4,aims,arguing,assurance
...,...,...,...
100,NaN,underdevelopment,NaN
101,NaN,unequal,NaN
102,NaN,unfortunately,NaN
103,NaN,unique,NaN


In [45]:
esg_new.to_csv(esg_new_path, index=False)